UN-FINISHED

In [3]:
import sys
sys.path.append('../wec-grid-code') # 
import pandas as pd
import wec_grid_class as wg
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
plt.rcParams["figure.figsize"] = (15,10)

    Sets PSSE environment to latest minor version among installed versions of PSSE 35.
    Use psse35.set_minor(n) to set PSSE35 minor version (n) to use.
        Example, for PSSE 35.0.x, use this as: psse35.set_minor(0)

 Input error detected at !
 -m ipykernel_launcher -f C:\Users\barajale\AppData\Roaming\jupyter\runtime\kernel-3fb68d58-0cad-4cdc-8ee2-bd9e2c76a7dd.json
                          !

 PSS(R)E Xplore Version 35
 Copyright (c) 1976-2022
 Siemens Industry, Inc.,
 Power Technologies International                            (PTI)
 This program is a confidential  unpublished  work  created  and  first
 licensed in 1976.  It is a trade secret which is the property of  PTI.
 All use,  disclosure,  and/or reproduction not specifically authorized
 by  PTI  is prohibited.   This  program is protected  under  copyright
 laws  of  non-U.S.  countries  and  by  application  of  international
 treaties.  All  Rights  Reserved  Under  The  Copyright  Laws.


           SIEMENS POWER T

In [4]:
# Read in case file 
case14 = r"../input_files/case14.raw"

In [5]:
# initalize WEC GRID Object
pf = wg.Wec_grid(case14)


 Reading IC, SBASE, REV, XFRRAT, NXFRAT, BASFRQ...

 Converting RAW data from PSS(R)E 33 to PSS(R)E 34 format
 Starting processing of Case identification data records
 Finished processing of 3 records for Case identification data
 Starting processing of Bus data records
 Finished processing of 14 records for Bus data
 Starting processing of Load data records
 Finished processing of 11 records for Load data
 Starting processing of Fixed shunt data records
 Finished processing of 1 records for Fixed shunt data
 Starting processing of Generator data records
 Finished processing of 5 records for Generator data
 Starting processing of Branch data records
 Finished processing of 17 records for Branch data
 Finished processing of 0 records for System switching device data
 Starting processing of Transformer data records
 Finished processing of 12 records for 3 Transformer data
 Starting processing of Area data records
 Finished processing of 0 records for Area data
 Starting processing of Tw

In [6]:
# Run first Power Flow and get bus values
pf.run_powerflow
pf.get_values(['BASE', 'PU', 'ANGLED', 'P', 'Q'])
print(pf.dataframe)

       Bus  BASE     PU     ANGLED  Type           P          Q
0    BUS 1   0.0  1.060   0.000000     3  232.346268 -16.759069
1    BUS 2   0.0  1.045  -4.980000     2   18.393555  29.763643
2    BUS 3   0.0  1.010 -12.719999     2  -94.275166   5.310905
3    BUS 4   0.0  1.019 -10.330000     1  -47.719318   8.118187
4    BUS 5   0.0  1.020  -8.780000     1   -7.731865  -2.865371
5    BUS 6   0.0  1.070 -14.220000     2  -11.000880   5.264521
6    BUS 7   0.0  1.062 -13.370001     1   -0.245777   0.234765
7    BUS 8   0.0  1.090 -13.360000     2    0.114721  17.326178
8    BUS 9   0.0  1.056 -14.939999     1  -29.305561   3.840285
9   BUS 10   0.0  1.051 -15.100001     1   -9.090860  -5.871971
10  BUS 11   0.0  1.057 -14.790001     1   -3.439518  -1.734084
11  BUS 12   0.0  1.055 -15.070000     1   -6.009285  -1.676000
12  BUS 13   0.0  1.050 -15.159998     1  -13.853797  -6.227429
13  BUS 14   0.0  1.036 -16.040001     1  -14.785610  -4.652751


In [5]:
# connect to Database and open in df
con = sqlite3.connect('../input_files/WEC_database.db')
cur = con.cursor()
wec_gen_values = pd.read_sql_query("SELECT * from WecOutput", con)
wec_gen_values

,Time,ibus,P
0,300.0,1.0,0.020204
1,600.0,1.0,0.013897
2,900.0,1.0,0.018099
3,1200.0,1.0,0.013920
4,1500.0,1.0,0.017861
5,1800.0,1.0,0.018624
6,2100.0,1.0,0.021036
7,2400.0,1.0,0.020560
8,2700.0,1.0,0.018935
9,3000.0,1.0,0.017833


In [15]:
def add_load_to(bus, load):
  """Create a load (can be negative) and add to bus.
     Returns load dictionary with new id ('id') and bus number ('i') set
  """
  newid = available_load_id(bus)
  psspy.load_data_3(
             bus,
             id=newid,
             realar1 =load.get('pl', 0),    # set PL [default 0]
             realar2 =load.get('ql', 0)    # set QL [default 0] 
  )
  load['id'] = newid
  load['i'] = bus
  return load

def available_load_id(bus):
  """Returns a random available load id for a bus."""
  taken_ids = load_ids_for_bus(bus)
  while True:
    newid = uuid.uuid4().hex[:2]
    if newid not in taken_ids: 
      break
  return newid


def load_ids_for_bus(target_bus):
  """returns existing load ids in a list for a bus."""
  buses, ierr = psspy.aloadint(
             -1,                # all buses
             flag=4,            # for all loads
             string=["NUMBER"]) # get bus number
  ids, ierr = psspy.aloadchar(
             -1,                # all buses
             flag=4,            # for all loads
             string=["ID"])     # get load id

  # converts [[101], [102]] and [['1'], ['2']] to [[101, '1'], [102, '2']]
  allbuses = zip(zip(*buses)[0], zip(*ids)[0])
  return [id for (bus, id) in allbuses if target_bus == bus]

In [25]:
ierr, buses = psspy.aloadint(
             -1,                # all buses
             flag=4,            # for all loads
             string="NUMBER")
buses

[[2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14]]

In [28]:
ierr, ids = psspy.aloadchar(
         -1,                # all buses
         flag=4,            # for all loads
         string=["ID"])     # get load id
ids

[['1 ', '1 ', '1 ', '1 ', '1 ', '1 ', '1 ', '1 ', '1 ', '1 ', '1 ']]

In [32]:
psspy.load_data_6(3, '1 ', realar2=20) # not sure what this does tbh


 No power flow data changed for load "1" at bus 3 [BUS 3             ]


0

In [2]:
pd.read_csv

 Volume in drive C is Windows
 Volume Serial Number is 3E6E-346B

 Directory of C:\Users\barajale\Desktop\research_code\tem\WEC-GRID\Jupyter Scripts

08/17/2022  11:23 AM    <DIR>          .
08/17/2022  11:23 AM    <DIR>          ..
08/17/2022  11:23 AM    <DIR>          .ipynb_checkpoints
08/17/2022  11:23 AM            19,888 PSSe_AC_injection.ipynb
08/17/2022  11:23 AM           259,565 PSSe_DC_injection.ipynb
               2 File(s)        279,453 bytes
               3 Dir(s)  366,023,786,496 bytes free
